## image captioning 
あんまりうまくいかなかった　　
結局画像の特徴量を見つける必要があるので難しそう

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

img_url = '/root/signate_tecno/input/test/Zzxnb7ogjzruKrw29pNvk3.jpg' 
raw_image = Image.open(img_url).convert('RGB')

# conditional image captioning
text = "a photography of a human dancing holding"
inputs = processor(raw_image, text, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
"""
# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
"""

## connected componentsをチャンネルとして追加したい

In [ ]:
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset,Subset, random_split
from sklearn.model_selection import KFold
import torchvision
from torchvision import datasets, models
from torchvision import transforms as T
import torchvision.transforms.functional as F
import torch.nn as nn
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt
from IPython.display import display
import lightning as L
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import torchmetrics, argparse

from torchvision.datasets import ImageFolder

from PIL import Image, ImageOps
import os

import multiprocessing
num_workers = multiprocessing.cpu_count()
print(num_workers)
import timm
import wandb
import segmentation_models_pytorch as smp
from ultralytics import YOLO
import cv2

In [33]:
# 画像が保存されているディレクトリ
input_dir = "/root/signate_tecno/input/test/"

# 画像ファイルのリストを取得
image_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.jpg')]

output_dir = "/root/signate_tecno/input/edge_test/"

In [27]:
# 画像を読み込む
for image_file in image_files:
    # 画像を読み込む。
    img = cv2.imread(image_file)

    # グレースケールに変換する。
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2値化する
    ret, bin_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # connected componentsを検出
    num_labels, labels_im = cv2.connectedComponents(bin_img)
    # 出力ファイル名を作成
    output_file = os.path.join(output_dir, os.path.basename(image_file))
    cv2.imwrite(output_file, labels_im)

In [34]:
# エッジ検出を行う関数を追加します

def detect_edges(image_path):
    # 画像を読み込む
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # エッジ検出を行う
    edges = cv2.Canny(image, 100, 200)
    
    return edges

for image_file in image_files:
    edges = detect_edges(image_file)
    output_file = os.path.join(output_dir, os.path.basename(image_file))
    cv2.imwrite(output_file, edges)

In [ ]:
edges.shape